In [10]:
import urllib
from lxml import html
from bs4 import BeautifulSoup
import datetime
import sys
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
import itertools
import StringIO
import matplotlib
import matplotlib.pyplot as plt
import subprocess
import os
%matplotlib inline

In [11]:
df = pd.read_csv('debate.csv')
df.head(n=5)

,Line,Speaker,Text,Date
0,1,Holt,Good evening from Hofstra University in Hempst...,9/26/16
1,2,Audience,(APPLAUSE),9/26/16
2,3,Clinton,"How are you, Donald?",9/26/16
3,4,Audience,(APPLAUSE),9/26/16
4,5,Holt,Good luck to you.,9/26/16


In [12]:
page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119181').read()
soup = BeautifulSoup(page, "lxml")
#speech = soup.find_all('span')[6]
speech = soup.find_all('p')
for sentence in speech:
    sentence = str(sentence)[3:-4]


In [20]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [22]:
valid_ids = range(165, 199)
for i in range(200,210):
    valid_ids.append(i)
trump_speeches = []
for valid_id in valid_ids:
    page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119' + str(valid_id)).read()
    soup = BeautifulSoup(page, "lxml")
    #print len(speech)
    #speech = soup.find_all('span')[6]
    speech = soup.find_all('p')
    for result in speech:
        result = str(result)[3:-4]
        for i, c in enumerate(result):
            if ord(c) < 0 or ord(c) > 127:
                result = result[:i] + '.' + result[i+1:]
        sentences = tokenizer.tokenize(result)
        for sent in sentences:
            trump_speeches.append(sent)

In [23]:
print len(trump_speeches)

5739


In [24]:
def removeInvalidChars(sentences):
    lst = []
    for i, sentence in enumerate(sentences):
        segs = sentence.split('<i>')
        lst.append(''.join([segs[i] for i in range(len(segs)) if i % 2 == 0]))
    return lst

In [25]:
trump_speeches = removeInvalidChars(trump_speeches)

In [26]:
valid_ids = range(148, 165)
for i in range(497,503):
    valid_ids.append(i)
for i in range(688,702):
    valid_ids.append(i)
clinton_speeches = []
for valid_id in valid_ids:
    page = urllib.urlopen('http://www.presidency.ucsb.edu/ws/index.php?pid=119' + str(valid_id)).read()
    soup = BeautifulSoup(page, "lxml")
    #speech = soup.find_all('span')[6]
    speech = soup.find_all('p')
    for result in speech:
        result = str(result)[3:-4]
        for i, c in enumerate(result):
            if ord(c) < 0 or ord(c) > 127:
                result = result[:i] + '.' + result[i+1:]
                #print result
        sentences = tokenizer.tokenize(result)
        for sent in sentences:
            clinton_speeches.append(sent)

In [27]:
clinton_speeches = removeInvalidChars(clinton_speeches)

In [28]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#fp = open("test.txt")
#data = fp.read()



In [29]:
trump_df = pd.DataFrame({}, columns = df.columns)
clinton_df = pd.DataFrame({}, columns = df.columns)
trump = []
clinton = []

end_t = 0
end_c = 0
for i in range(df.shape[0]):
#for i in range(5):
    if df['Speaker'].values[i] == 'Trump':
        speech = df['Text'].values[i]
        sentences = tokenizer.tokenize(speech)
        for sentence in sentences:
            trump_df.loc[end_t] = [end_t, df['Speaker'].values[i], sentence, df['Date'].values[i]]
            trump.append(sentence)
            end_t += 1
    if df['Speaker'].values[i] == 'Clinton':
        speech = df['Text'].values[i]
        sentences = tokenizer.tokenize(speech)
        for sentence in sentences:
            clinton_df.loc[end_c] = [end_c, df['Speaker'].values[i], sentence, df['Date'].values[i]]
            clinton.append(sentence)
            end_c += 1
            
        
        
    

In [30]:
print clinton_speeches[:20]

['I want to recognize the Institute for all you do to inspire the next generation of Latino leaders.', 'Last year, I had the chance to spend time with some of the CHCI interns and fellows.', 'It really was like seeing the future of America in one room.', "I can't wait to see everything that they achieve.", "And I can't wait, if I'm fortunate enough to be president, to put some of them to work! ", 'And I want to thank all my friends in the Congressional Hispanic Caucus.', 'You fight every single day to lift up the Latino community ... when the cameras are rolling and when the cameras are off, at home in your districts and here in Washington.', "And no one understands better than you the pivotal moment we're at right now ... not just for Latinos but for our country.", 'My friends in the caucus have traveled to every battleground state, registered voters.', "You've stayed focused no matter what kind of outlandish and offensive comments we have heard from my opponent and his supporters.", 

In [31]:
print len(trump)

2020


In [32]:
new_word = "NEW"
begin_sentence = "BEGIN"
end_sentence = "END"
for i in trump_speeches:
    trump.append(i)


In [33]:
print len(trump)

7759


In [34]:
for i in clinton_speeches:
    clinton.append(i)

In [35]:
print len(clinton)

8406


In [36]:
name = 'astro_sentences.txt'
f = open(name,'w')
'''
for sentence in trump:
    f.write(sentence+',0\n')
for sentence in clinton:
	f.write(sentence + ',1\n')
'''
tot = 0
t = 0
c = 0
cont = True
while cont:
    if np.random.uniform(low=0.0, high=1.0) < .5:
        f.write(trump[t] + ',0\n')
        tot += 1
        t += 1
    else:
        f.write(clinton[c] + ',1\n')
        tot += 1
        c += 1
    if t == len(trump):
        while c < len(clinton):
            f.write(clinton[c] + ',1\n')
            tot += 1
            c += 1
        cont = False
    elif c == len(clinton):
        while t < len(trump):
            f.write(trump[t] + ',0\n')
            tot += 1
            t += 1
        cont = False

f.close()
print tot

16165


In [44]:
print len(trump)

7759


In [45]:

trump = ["%s %s %s" % (begin_sentence, s, end_sentence) for s in trump]

trump_words = [nltk.word_tokenize(sent) for sent in trump]
#clinton_words = [nltk.word_tokenize(sent) for sent in clinton]
print type(trump)

<type 'list'>


In [46]:
word_freq = nltk.FreqDist(itertools.chain(*trump_words))

In [48]:
vocab_size = 2200
new_word = "NEW"
begin_sentence = "BEGIN"
end_sentence = "END"

vocab = word_freq.most_common(vocab_size-1)

index_to_word = [w[0] for w in vocab]
index_to_word.append(new_word)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

In [78]:

for i, sent in enumerate(trump_words):
    trump_words[i] = [w if w in word_to_index else new_word for w in sent]
    
    
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in trump_words])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in trump_words])

In [79]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [80]:
class RNN:
     
    def __init__(self, word_d, hidden_d=100, bptt_end=4):
        self.word_d = word_d
        self.bptt_end = bptt_end
        self.hidden_d = hidden_d
        
        self.U = np.random.uniform(-np.sqrt(1./word_d), np.sqrt(1./word_d), (hidden_d, word_d))
        self.V = np.random.uniform(-np.sqrt(1./hidden_d), np.sqrt(1./hidden_d), (word_d, hidden_d))
        self.W = np.random.uniform(-np.sqrt(1./hidden_d), np.sqrt(1./hidden_d), (hidden_d, hidden_d))


In [81]:
def fwd_propagate(self, x):
    total_time = len(x)
    s = np.zeros((total_time + 1, self.hidden_d))
    s[-1] = np.zeros(self.hidden_d)
    o = np.zeros((total_time, self.word_d))
    for t in range(total_time):
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]
 
RNN.fwd_propagate = fwd_propagate

In [82]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    return np.argmax(self.fwd_propagate(x)[0], axis = 1)
 
RNN.predict = predict

In [83]:
model = RNN(vocab_size)
o, s = model.fwd_propagate(X_train[10])
print o.shape
print o

(30, 2200)
[[ 0.0004566   0.00045177  0.00045222 ...,  0.0004606   0.00045072
   0.00045805]
 [ 0.00045548  0.00045312  0.00045196 ...,  0.00045784  0.00044956
   0.00045599]
 [ 0.0004525   0.00045689  0.0004547  ...,  0.00044912  0.00045292
   0.00045084]
 ..., 
 [ 0.00045087  0.00045228  0.00045041 ...,  0.00045097  0.00045611
   0.00045182]
 [ 0.00044937  0.00045358  0.00045047 ...,  0.00045237  0.0004638
   0.00045149]
 [ 0.0004564   0.00045696  0.00045649 ...,  0.00046163  0.00045309
   0.00045458]]


In [84]:
predictions = model.predict(X_train[10])
print predictions

[1480 1480 1089 1279  508 1787  380  453 1555 1740 1508 1899 1903   70 1508
 1780 1073  537 1901 2139 1460 1145 1192  729 1473 1508 1760  719  848 1296]


In [85]:
def loss_tot(self, x, y):
    loss = 0
    for i in range(len(y)):
        o, s = self.fwd_propagate(x[i])
        accurate_pred = o[range(len(y[i])), y[i]]
        loss += -1 * np.sum(np.log(accurate_pred))
    return loss
 
def loss(self, x, y):
    tot = np.sum([len(yi) for yi in y])
    return self.loss_tot(x,y) / tot
 
RNN.loss_tot = loss_tot
RNN.loss = loss

In [86]:
print "Expected Loss: %f" % np.log(vocab_size)
print "Empirical loss: %f" % model.loss(X_train, y_train)

Expected Loss: 7.696213
Empirical loss: 7.695186


In [87]:
def bwd_propagate(self, x, y):
    total_time = len(y) 
    o, s = self.fwd_propagate(x)

    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    
    delta_o = o
    delta_o[range(len(y)), y] -= 1.
    
    for t in reversed(range(total_time)):
        dLdV += np.outer(delta_o[t], s[t].T)
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        a = max(0, t-self.bptt_end)
        for step in reversed(range(a, t+1)):
            dLdW += np.outer(delta_t, s[step-1])              
            dLdU[:,x[step]] += delta_t
            delta_t = (1 - (s[step-1] ** 2)) * self.W.T.dot(delta_t)
    return [dLdU, dLdV, dLdW]
 
RNN.bwd_propagate = bwd_propagate

In [88]:

def calc_grad(self, x, y, learning_rate):
    # Find gradients
    dLdU, dLdV, dLdW = self.bwd_propagate(x, y)
    
    # Update parameters
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW
 
RNN.calc_grad = calc_grad



In [94]:
def sgd_params(model, X_train, y_train, 
                   learning_rate=0.005, 
                   nepoch=100, 
                   evaluate_cycle=5):
    losses = []
    num_seen = 0
    for epoch in range(nepoch):
        if (epoch % evaluate_cycle == 0):
            # evaluate
            loss = model.loss(X_train, y_train)
            losses.append((num_seen, loss))
            print "Loss after seen=%d epoch=%d: %f" % (num_seen, epoch, loss)
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5 
            sys.stdout.flush()

        for i in range(len(y_train)):
            model.calc_grad(X_train[i], y_train[i], learning_rate)
            num_seen += 1

In [96]:
model = RNN(vocab_size)
losses = sgd_params(model, X_train, y_train, nepoch=1, evaluate_cycle=1)



Loss after seen=0 epoch=0: 7.696420


In [100]:
def generate_text(model):
    sent = [word_to_index[begin_sentence]]
    while not sent[-1] == word_to_index[end_sentence]:
        prob_word = model.fwd_propagate(sent)[0]
        sampled_word = word_to_index[new_word]
        
        while sampled_word == word_to_index[new_word]:
            samples = np.random.multinomial(1, prob_word[-1])
            sampled_word = np.argmax(samples)
        sent.append(sampled_word)

    sent_str = [index_to_word[x] for x in sent[1:-1]]
    return sent_str
 


In [101]:
sentences = 8
min_len = 7
 
for i in range(sentences):
    sent = []

    while len(sent) < min_len:
        sent = generate_text(model)
    print " ".join(sent)

BEGIN American now , soul close up for countries , and far the families .
BEGIN This there going to support is and trillion that will deserves work and America federal do .
BEGIN caused are going to over percent November , .
BEGIN Just will will that paying country days streets and the good-paying Post in scandal every save American child Hillary and People for tired .
BEGIN We will am back taxes moms years being family States of last supporting and our cartels , absolute of the long through into a 30 .
BEGIN She , at used my 45 gun saved , and , I will going looked have and lift and over for people to together citizens jobs .
BEGIN She many been a buy Again jobs out , Root to going murders employers be 'm .
BEGIN We are Make America trade your are trade laughing our trade , us will not to inner America .
